In [1]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
import geocoder
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors


In [2]:
df=pd.read_csv('data1.csv')
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653340,-79.509766
99,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.648700,-79.385450
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.632798,-79.493017


In [3]:
Address='Toronto'
g=geocoder.arcgis(Address)
cord=g.latlng
map_Toronto=folium.Map(location=cord,zoom_start=10)
for lat,lng,burough,neighborhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label='{},{}'.format(neighborhood,burough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [4]:
CLIENT_ID = 'FUMSJUVTBWTT4XGJQNS4NJSJGVY4MF0ISWGVN0DRIDQB0EQI' 
CLIENT_SECRET = 'THZ2C1RIWKLE3EDNW5M3FRB1GUFTTCLABEWHOUXSIGKJYHZZ' 
VERSION = '20180605' 
LIMIT=100

In [5]:
def getNearbyVenues(names,latitudes,longitudes,radius=500):
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.728102,-79.311890,Tim Hortons,43.725517,-79.313103,Coffee Shop
3,Victoria Village,43.728102,-79.311890,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.728102,-79.311890,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection


In [7]:
venue_onehot=[]
venue_onehot=pd.get_dummies(venues[['Venue Category']],prefix="",prefix_sep="")
venue_onehot['Neigh']=venues['Neighborhood']
fixed_columns = [venue_onehot.columns[-1]] + list(venue_onehot.columns[:-1])
venue_onehot = venue_onehot[fixed_columns]
venue_onehot.head()

,Neigh,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
venue_grouped=venue_onehot.groupby('Neigh').mean().reset_index()
venue_grouped.head()

,Neigh,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood , Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park , Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [10]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = venue_grouped['Neigh']

for ind in np.arange(venue_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(venue_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Sushi Restaurant,Skating Rink,Badminton Court,Supermarket,Shanghai Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,"Alderwood , Long Branch",Convenience Store,Pub,Gas Station,Coffee Shop,Dance Studio,Gym,Pharmacy,Pizza Place,Sandwich Place,Field
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Pharmacy,Shopping Mall,Bridal Shop,Middle Eastern Restaurant,Sandwich Place,Fried Chicken Joint,Supermarket,Sushi Restaurant
3,Bayview Village,Construction & Landscaping,Trail,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Field,Dumpling Restaurant
4,"Bedford Park , Lawrence Manor East",Italian Restaurant,Thai Restaurant,Coffee Shop,Sandwich Place,Restaurant,Juice Bar,Pub,Japanese Restaurant,Liquor Store,Indian Restaurant


In [11]:
k=5
venue_cluster=venue_grouped.drop('Neigh',axis=1)
kmean=KMeans(n_clusters=k,random_state=0).fit(venue_cluster)
kmean.labels_[0:10]

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [12]:
venues_sorted.insert(0,'Cluster Labels',kmean.labels_)

In [13]:
venues_merged=df
venues_merged=venues_merged.join(venues_sorted.set_index('Neighborhood'),on='Neighborhood')
venues_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,4.0,Food & Drink Shop,Park,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
1,M4A,North York,Victoria Village,43.728102,-79.311890,1.0,Park,Pizza Place,Portuguese Restaurant,Coffee Shop,Intersection,Yoga Studio,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041,1.0,Pub,Café,Athletics & Sports,Mediterranean Restaurant,Distribution Center,Music Venue,Chocolate Shop,Bank,Tech Startup,Thai Restaurant
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211,1.0,Clothing Store,Cosmetics Shop,Furniture / Home Store,Bookstore,Sporting Goods Shop,Food Court,Men's Store,Restaurant,Sushi Restaurant,American Restaurant
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390,1.0,Coffee Shop,Café,Sandwich Place,Juice Bar,Bookstore,Fried Chicken Joint,Burger Joint,Burrito Place,Middle Eastern Restaurant,Yoga Studio


In [14]:
venues_merged.drop(venues_merged[venues_merged['Cluster Labels'].isnull()].index,inplace=True)
venues_merged['Cluster Labels']=venues_merged['Cluster Labels'].astype(int)
venues_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,4,Food & Drink Shop,Park,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
1,M4A,North York,Victoria Village,43.728102,-79.311890,1,Park,Pizza Place,Portuguese Restaurant,Coffee Shop,Intersection,Yoga Studio,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041,1,Pub,Café,Athletics & Sports,Mediterranean Restaurant,Distribution Center,Music Venue,Chocolate Shop,Bank,Tech Startup,Thai Restaurant
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211,1,Clothing Store,Cosmetics Shop,Furniture / Home Store,Bookstore,Sporting Goods Shop,Food Court,Men's Store,Restaurant,Sushi Restaurant,American Restaurant
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390,1,Coffee Shop,Café,Sandwich Place,Juice Bar,Bookstore,Fried Chicken Joint,Burger Joint,Burrito Place,Middle Eastern Restaurant,Yoga Studio


In [15]:
map = folium.Map(location=cord, zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venues_merged['Latitude'], venues_merged['Longitude'], venues_merged['Neighborhood'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map)
       
map

In [16]:
venues_merged.loc[venues_merged['Cluster Labels'] == 0, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0,Trail,Yoga Studio,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Field,Dumpling Restaurant
26,Scarborough,0,Construction & Landscaping,Lounge,Trail,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Eastern European Restaurant
39,North York,0,Construction & Landscaping,Trail,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Field,Dumpling Restaurant


In [17]:
venues_merged.loc[venues_merged['Cluster Labels'] == 1, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Park,Pizza Place,Portuguese Restaurant,Coffee Shop,Intersection,Yoga Studio,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm
2,Downtown Toronto,1,Pub,Café,Athletics & Sports,Mediterranean Restaurant,Distribution Center,Music Venue,Chocolate Shop,Bank,Tech Startup,Thai Restaurant
3,North York,1,Clothing Store,Cosmetics Shop,Furniture / Home Store,Bookstore,Sporting Goods Shop,Food Court,Men's Store,Restaurant,Sushi Restaurant,American Restaurant
4,Downtown Toronto,1,Coffee Shop,Café,Sandwich Place,Juice Bar,Bookstore,Fried Chicken Joint,Burger Joint,Burrito Place,Middle Eastern Restaurant,Yoga Studio
7,North York,1,Construction & Landscaping,Athletics & Sports,Bank,Restaurant,Other Great Outdoors,Park,Trail,Burger Joint,Field,Filipino Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Bar,Japanese Restaurant,Asian Restaurant,Seafood Restaurant
99,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Pub,Café,Pizza Place,Men's Store,Mediterranean Restaurant,Sushi Restaurant
100,East Toronto,1,Coffee Shop,Hotel,Bar,Restaurant,Italian Restaurant,American Restaurant,Asian Restaurant,Café,Taco Place,Sushi Restaurant
101,Etobicoke,1,Construction & Landscaping,Home Service,Bank,Park,Fast Food Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Field


In [18]:
venues_merged.loc[venues_merged['Cluster Labels'] == 2, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Scarborough,2,Auto Garage,Yoga Studio,Field,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Eastern European Restaurant


In [19]:
venues_merged.loc[venues_merged['Cluster Labels'] == 3, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York,3,Bus Stop,Playground,Yoga Studio,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant
57,North York,3,Playground,Park,Yoga Studio,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Field
73,Central Toronto,3,Gym Pool,Garden,Playground,Park,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
91,Downtown Toronto,3,Park,Playground,Candy Store,Grocery Store,Yoga Studio,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market


In [20]:
venues_merged.loc[venues_merged['Cluster Labels'] == 4, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Food & Drink Shop,Park,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
5,Etobicoke,4,Park,Baseball Field,Skating Rink,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Yoga Studio
27,North York,4,Dog Run,Park,Residential Building (Apartment / Condo),Farmers Market,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Fast Food Restaurant,Dumpling Restaurant
49,North York,4,Park,Bakery,Basketball Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Electronics Store
66,North York,4,Convenience Store,Park,Bank,Creperie,Cuban Restaurant,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop
68,Central Toronto,4,Park,Yoga Studio,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
88,Etobicoke,4,Park,Yoga Studio,Convenience Store,Grocery Store,Skating Rink,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market
98,Etobicoke,4,Lounge,Park,Yoga Studio,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Eastern European Restaurant
